In [3]:
#importing library
import pandas as pd
import numpy as np
import os

# Path

In [4]:
path = r'/Users/mehreenbecker/4.3 IC Data Import and Descriptive Analysis/Original File'

# Data

In [5]:
ords_prods_merge = pd.read_pickle(os.path.join(path, 'ords_prods_merge.pkl'))

# Checking the Data

In [6]:
ords_prods_merge.head()

product_id                product_name  aisle_id  department_id  prices  \
0           1  Chocolate Sandwich Cookies        61             19     5.8   
1           1  Chocolate Sandwich Cookies        61             19     5.8   
2           1  Chocolate Sandwich Cookies        61             19     5.8   
3           1  Chocolate Sandwich Cookies        61             19     5.8   
4           1  Chocolate Sandwich Cookies        61             19     5.8   

   Unnamed: 0  order_id  user_id  order_number  orders_day_of_week  \
0        1987   3139998      138            28                   6   
1        1989   1977647      138            30                   6   
2       11433    389851      709             2                   0   
3       12198    652770      764             1                   3   
4       12200   1813452      764             3                   4   

   order_hour_of_day  days_since_prior_order  First_order  add_to_cart_order  \
0                 11                     3.0        False                  5   
1                 17                    20.0        False                  1   
2                 21                     6.0        False                 20   
3                 13                     NaN         True                 10   
4                 17                     9.0        False                 11   

   reordered _merge  
0          0   both  
1          1   both  
2          0   both  
3          0   both  
4          1   both

In [7]:
df = ords_prods_merge.iloc[:1_000_000].copy()

In [8]:
df.shape

(1000000, 16)

# Price Label

In [9]:
def price_label(row):
    if row['prices'] <= 5:
        return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15):
        return 'Mid-range product'
    elif row['prices'] > 15:
        return 'High-range'
    else:
        return 'Not enough data'

In [10]:
df['price_range'] = df.apply(price_label, axis=1)

In [11]:
df['price_range'].value_counts(dropna=False)

price_range
Mid-range product    652638
Low-range product    338018
High-range             9344
Name: count, dtype: int64

In [12]:
df['prices'].max()

24.5

In [13]:
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product'
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [14]:
df['price_range_loc'].value_counts(dropna=False)

price_range_loc
Mid-range product     652638
Low-range product     338018
High-range product      9344
Name: count, dtype: int64

In [15]:
ords_prods_merge.loc[ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'
ords_prods_merge.loc[(ords_prods_merge['prices'] <= 15) & (ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'
ords_prods_merge.loc[ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [16]:
ords_prods_merge['price_range_loc'].value_counts(dropna=False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: count, dtype: int64

In [17]:
ords_prods_merge['prices'].max()

99999.0

In [18]:
print(ords_prods_merge.columns.tolist())

['product_id', 'product_name', 'aisle_id', 'department_id', 'prices', 'Unnamed: 0', 'order_id', 'user_id', 'order_number', 'orders_day_of_week', 'order_hour_of_day', 'days_since_prior_order', 'First_order', 'add_to_cart_order', 'reordered', '_merge', 'price_range_loc']


# Orders Day of Week

In [19]:
df ['orders_day_of_week'].value_counts(dropna=False)

orders_day_of_week
0    183956
1    174070
6    137125
5    132019
2    132015
3    120883
4    119932
Name: count, dtype: int64

## 0 - Saturday is the busiest day; 4 - Wednesday is the least busy day

In [21]:
result = []
for value in df ['orders_day_of_week']:
    if value == 0:
        result.append('Busiest day')
    elif value == 4:
        result.append('Least busy')
    else:
        result.append('Regularly busy')

df ['busiest_day'] = result

In [22]:
df ['busiest_day'].value_counts(dropna=False)

busiest_day
Regularly busy    696112
Busiest day       183956
Least busy        119932
Name: count, dtype: int64

In [23]:
results = []
for value in df ['orders_day_of_week']:
    if value in (0, 1):
        results.append('Busiest days')
    elif value in (3, 4):
        results.append('Slowest days')
    else:
        results.append('Regular days')

df ['busiest_days'] = results

In [24]:
df ['busiest_days'].value_counts(dropna=False)

busiest_days
Regular days    401159
Busiest days    358026
Slowest days    240815
Name: count, dtype: int64

In [25]:
df ['order_hour_of_day'].value_counts(dropna=False).sort_index()

order_hour_of_day
0      6560
1      3576
2      2174
3      1653
4      1639
5      2831
6      9090
7     27542
8     53294
9     76870
10    86241
11    84780
12    80541
13    82272
14    83070
15    81234
16    77784
17    64196
18    50082
19    38584
20    30367
21    24338
22    19126
23    12156
Name: count, dtype: int64

## Most orders are between = 9–16; average orders = 7–8 and 17–22; fewest order = everything else 

In [26]:
results_hours = []
for value in df['order_hour_of_day']:
    if value in (9, 10, 11, 12, 13, 14, 15, 16):
        results_hours.append('Most orders')
    elif value in (7, 8, 17, 18, 19, 20, 21, 22):
        results_hours.append('Average orders')
    else:
        results_hours.append('Fewest orders')

df ['busiest_period_of_day'] = results_hours

In [27]:
pd.read_csv(os.path.join(path, 'orders_cleaned.csv')).head()

Unnamed: 0.1  Unnamed: 0  order_id  user_id  order_number  \
0             0           0   2539329        1             1   
1             1           1   2398795        1             2   
2             2           2    473747        1             3   
3             3           3   2254736        1             4   
4             4           4    431534        1             5   

   orders_day_of_week  order_hour_of_day  days_since_prior_order  First_order  
0                   2                  8                     NaN         True  
1                   3                  7                    15.0        False  
2                   3                 12                    21.0        False  
3                   4                  7                    29.0        False  
4                   4                 15                    28.0        False

In [28]:
df_csv = pd.read_csv(os.path.join(path, 'orders_cleaned.csv'), usecols=["orders_day_of_week", "order_hour_of_day"])

In [30]:
ords_prods_merge = pd.merge(
    ords_prods_merge,
    df,
    on="order_id",
    how="left"
)

# Checking Data

In [31]:
ords_prods_merge.head()

product_id_x              product_name_x  aisle_id_x  department_id_x  \
0             1  Chocolate Sandwich Cookies          61               19   
1             1  Chocolate Sandwich Cookies          61               19   
2             1  Chocolate Sandwich Cookies          61               19   
3             1  Chocolate Sandwich Cookies          61               19   
4             1  Chocolate Sandwich Cookies          61               19   

   prices_x  Unnamed: 0_x  order_id  user_id_x  order_number_x  \
0       5.8          1987   3139998        138              28   
1       5.8          1989   1977647        138              30   
2       5.8         11433    389851        709               2   
3       5.8         11433    389851        709               2   
4       5.8         11433    389851        709               2   

   orders_day_of_week_x  ...  days_since_prior_order_y  First_order_y  \
0                     6  ...                       3.0          False   
1                     6  ...                      20.0          False   
2                     0  ...                       6.0          False   
3                     0  ...                       6.0          False   
4                     0  ...                       6.0          False   

   add_to_cart_order_y  reordered_y  _merge_y        price_range  \
0                  5.0          0.0      both  Mid-range product   
1                  1.0          1.0      both  Mid-range product   
2                 20.0          0.0      both  Mid-range product   
3                 25.0          0.0      both  Low-range product   
4                 16.0          0.0      both  Mid-range product   

   price_range_loc_y     busiest_day  busiest_days  busiest_period_of_day  
0  Mid-range product  Regularly busy  Regular days            Most orders  
1  Mid-range product  Regularly busy  Regular days         Average orders  
2  Mid-range product     Busiest day  Busiest days         Average orders  
3  Low-range product     Busiest day  Busiest days         Average orders  
4  Mid-range product     Busiest day  Busiest days         Average orders  

[5 rows x 37 columns]

# Merging to Pickle

In [32]:
ords_prods_merge.to_pickle(
    os.path.join(path, 'ords_prods_merge_extra_columns.pkl')
)